<a href="https://colab.research.google.com/github/sagunkayastha/Cell_Seg_count/blob/master/SkIn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install efficientnet_pytorch
!cp /content/drive/My\ Drive/Datasets/skin_data.zip ./
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!unzip -q skin_data.zip -d data

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-9lbghjjp
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-9lbghjjp
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=67247 sha256=72d762820c551256037c3ddf57d058634ad6ac125f6931e8d405b090ae0f81c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ub59rgvs/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
Successfully built albumentations
  Found existing installation: albumentations 0.4.6
    Uninstalling albumentations-0.4.6:
      Successfully uninstalled albumentations-0.4.6
replace data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A


In [3]:
import pandas as pd
import os
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import cv2
from torchvision import transforms as T
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
from efficientnet_pytorch import EfficientNet
from torchvision import models
import pdb
import albumentations as A
from albumentations.pytorch.transforms import ToTensor
import matplotlib.pyplot as plt
from fastprogress.fastprogress import master_bar, progress_bar
from sklearn import model_selection
import time
import albumentations

In [4]:
data_path = '/content/data'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Batch_size = 32
image_size = 216

In [5]:
df = pd.read_csv(os.path.join(data_path,'train.csv'))
df["kfold"] = -1    
df = df.sample(frac=1).reset_index(drop=True)
y = df.target.values
kf = model_selection.StratifiedKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f

df.to_csv("train_folds.csv", index=False)
df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,tfrecord,width,height,kfold
0,ISIC_3394796,IP_6767827,female,40.0,torso,unknown,benign,0,13,640,480,0
1,ISIC_9274041,IP_0612651,female,25.0,upper extremity,unknown,benign,0,5,6000,4000,0
2,ISIC_1241680,IP_0718832,female,55.0,torso,unknown,benign,0,10,1872,1053,0
3,ISIC_3455749,IP_0660290,male,60.0,lower extremity,unknown,benign,0,10,6000,4000,0
4,ISIC_1672458,IP_9927968,male,35.0,head/neck,unknown,benign,0,4,640,480,0


In [6]:
fold=0
df_train = df[df.kfold != fold].reset_index(drop=True)
df_valid = df[df.kfold == fold].reset_index(drop=True)
len(df_train),len(df_valid)

(26500, 6626)

In [7]:
class ScancerDataset(Dataset):
    def __init__(self, df, data_folder, transforms=None):
        self.df = df
        self.data_folder = data_folder
        self.transforms_object = transforms
        
    def __getitem__(self, idx):
        img_name = self.df.loc[idx,'image_name']
        img_path = os.path.join(self.data_folder,img_name+'.jpg')
        image_data = Image.open(img_path)
        
        if self.transforms_object:
            image_data = self.transforms_object(image_data)
        if 'target' in self.df.columns.values:
            y = self.df.loc[idx,'target']
        else :
            y = 1
        return image_data ,torch.tensor([target],dtype=torch.float32)
        
    def __len__(self):
        return self.df.shape[0]

In [8]:
effic = EfficientNet.from_pretrained('efficientnet-b1')
def GlobalAveragePooling(x):
    return x.mean(axis=-1).mean(axis=-1)

class Scancer(nn.Module):
    def __init__(self, efn):
        super(Scancer, self).__init__()
        # self.dense_output = nn.Linear(features, 1)
        self.efn = efn
        self.efn._fc = nn.Linear(in_features=1280,out_features=512, bias=True)
        
        self.model= nn.Sequential(self.efn,
                                     nn.Linear(512,128),
                                     nn.LeakyReLU(),
                                     nn.Linear(128,16),
                                     nn.LeakyReLU(),
                                     nn.Linear(16,1))
        
        
    def forward(self, x):
        output = self.model(x)
        return output


Loaded pretrained weights for efficientnet-b1


In [9]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
        callback_get_label func: a callback-like function which takes two arguments - dataset and index
    """

    def __init__(self, dataset, indices=None, num_samples=None, callback_get_label=None):

        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices

        # define custom callback
        self.callback_get_label = callback_get_label

        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples

        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1

        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):  
        return dataset.train_labels[idx].item()

    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

## AUG

In [10]:
##https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/159476
## Roman microscope aug
class Microscope:
    def __init__(self, p: float = 0.5):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8),
                        (img.shape[0]//2, img.shape[1]//2),
                        random.randint(img.shape[0]//2 - 3, img.shape[0]//2 + 15),
                        (0, 0, 0),
                        -1)

            mask = circle - 255
            img = np.multiply(img, mask)

        return img

    def __repr__(self):
        return f'{self.__class__.__name__}(p={self.p})'

In [11]:
train_transform = T.Compose([
    T.RandomResizedCrop(size=256,scale=(0.7,1)),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

train_dataset = ScancerDataset(df_train,os.path.join(data_path,'train'),transforms=train_transform )
valid_dataset = ScancerDataset(df_valid,os.path.join(data_path,'train'),transforms=train_transform )
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Batch_size, shuffle=True, num_workers=4
    )
valid_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Batch_size, shuffle=False, num_workers=4
    )
print(len)

<built-in function len>


## Model


In [12]:
# https://arxiv.org/abs/1708.02002
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

### TEST 1


In [11]:

s_model= Scancer(effic)
s_model.to(device)
criterion = nn.BCEWithLogitsLoss()

def bce(y_true, y_pred):
    return nn.BCEWithLogitsLoss()(y_pred, y_true)

def acc(y_true, y_pred):
    y_true = y_true.squeeze()
    y_pred = nn.Sigmoid()(y_pred).squeeze()
    return (y_true == torch.round(y_pred)).float().sum()/len(y_true)

In [51]:
train_batches = len(train_loader) - 1
EPOCHS=10
mb = master_bar(range(EPOCHS))
mb.write(['epoch','train_loss'],table=True)

for epoch in range(EPOCHS):
    print(("EPOCH %s" + str(epoch+1) + "%s") )
    
    batch = 1
    s_model.train()
    trn_loss,val_loss = 0.0,0.0
    for train_batch in  train_loader:
        train_img, train_targ = train_batch
        train_targ = train_targ.view(-1, 1)
        train_img, train_targ = train_img.to(device), train_targ.to(device)
        
        if batch >= train_batches: break
        train_preds = s_model.forward(train_img)
        train_acc = acc(train_targ, train_preds)
        train_loss = bce(train_targ, train_preds)
            
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        trn_loss +=train_loss
        
        if batch%100==0:
          print(train_loss/Batch_size,train_acc)
        batch = batch + 1
        # print(batch)  
    loss =trn_loss/ len(train_loader)  
    
    print(loss)
        # batch = batch + 1
        # accuracy = np.round(train_acc.item(), 3)
        # print_metric(accuracy, batch, 0, start, end, metric="acc", typ="Train")
        
            
    # network.eval()
    # val_loss, val_acc, val_points = 0, 0, 0
        
    # with torch.no_grad():
    #     for val_batch in tqdm(val_loader):
    #         val_img, val_targ = val_batch
    #         val_targ = val_targ.view(-1, 1)
    #         val_img, val_targ = val_img.to(device), val_targ.to(device)

    #         val_points += len(val_targ)
    #         val_preds = s_model.forward(val_img)
    #         val_acc += acc(val_targ, val_preds).item()*len(val_preds)
    #         val_loss += bce(val_targ, val_preds).item()*len(val_preds)
        
    
    # val_acc /= val_points
    # val_loss /= val_points
    # accuracy = np.round(val_acc, 3)
    # # print_metric(accuracy, 0, epoch, start, end, metric="acc", typ="Val")
    
    print("")

print("ENDING TRAINING ...")

EPOCH %s1%s
tensor(0.0002, device='cuda:0', grad_fn=<DivBackward0>) tensor(1., device='cuda:0')
tensor(0.0010, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.9844, device='cuda:0')
tensor(0.0003, device='cuda:0', grad_fn=<DivBackward0>) tensor(1., device='cuda:0')
tensor(0.0020, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.9688, device='cuda:0')
tensor(0.0859, device='cuda:0', grad_fn=<DivBackward0>)

EPOCH %s2%s
tensor(0.0011, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.9844, device='cuda:0')
tensor(0.0032, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.9531, device='cuda:0')
tensor(0.0021, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.9688, device='cuda:0')
tensor(0.0013, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.9844, device='cuda:0')
tensor(0.0862, device='cuda:0', grad_fn=<DivBackward0>)

EPOCH %s3%s
tensor(0.0011, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.9844, device='cuda:0')
tensor(0.0003, device='cuda:0', grad_fn=<DivBackward0>) tensor(1.,

### Test 2


In [13]:
class ScancerDataset(Dataset):
    def __init__(self, df, data_folder, transforms=None):
        self.df = df
        self.data_folder = data_folder
        self.transform = transforms
        
    def __getitem__(self, idx):
        img_name = self.df.loc[idx,'image_name']
        img_path = os.path.join(self.data_folder,img_name+'.jpg')
        
        image = cv2.imread(img_path )
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image'].astype(np.float32)
        else:
            image = image.astype(np.float32)

        image = image.transpose(2, 0, 1)
        if 'target' in self.df.columns.values:
            y = self.df.loc[idx,'target']
        else :
            y = 1
        return torch.tensor(image).float(),torch.tensor([y],dtype=torch.float32)
        
    def __len__(self):
        return self.df.shape[0]

In [80]:
Batch_size=64
train_transform = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.2, p=0.75),
        albumentations.RandomContrast(limit=0.2, p=0.75),
        albumentations.OneOf([
            albumentations.MotionBlur(blur_limit=5),
            albumentations.MedianBlur(blur_limit=5),
            albumentations.GaussianBlur(blur_limit=5),
            albumentations.GaussNoise(var_limit=(5.0, 30.0)),
        ], p=0.7),

        albumentations.OneOf([
            albumentations.OpticalDistortion(distort_limit=1.0),
            albumentations.GridDistortion(num_steps=5, distort_limit=1.),
            albumentations.ElasticTransform(alpha=3),
        ], p=0.7),

        albumentations.CLAHE(clip_limit=4.0, p=0.7),
        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
        albumentations.Normalize()
    ])


transforms_val = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize()
])

train_dataset = ScancerDataset(df_train,os.path.join(data_path,'train'),transforms=train_transform )
valid_dataset = ScancerDataset(df_valid,os.path.join(data_path,'train'),transforms=train_transform )
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Batch_size, shuffle=True, num_workers=4
    )
valid_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Batch_size, shuffle=False, num_workers=4
    )
print(len(train_loader))

415


/usr/local/lib/python3.6/dist-packages/albumentations/augmentations/transforms.py:2552: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [ ]:
model= Scancer(effic)
model.to(device)
criterion = nn.BCEWithLogitsLoss()

epochs = 10

optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
for epoch in range(epochs):
  
      start_time = time.time()
      correct = 0
      running_loss = 0
      total_predictions = 0.0
      model.train()
      count=0
      for train_batch in train_loader:
          optimizer.zero_grad()
          
          train_img, train_targ = train_batch
          
          x,y = train_img.float().to(device), train_targ.to(device)
          z = model(x)

          
          loss = criterion(z, y.reshape(-1,1).float())
          loss.backward()
          optimizer.step()
          pred = torch.round(torch.sigmoid(z))  # round off sigmoid to obtain predictions
          total_predictions += y.size(0)
          correct  += (y.squeeze().cpu() ==pred.squeeze().cpu()).sum().item()
          
          running_loss += loss.item()
          
          if count%40==0:
            print(count)
          count+=1
          #   test_loss = 0
          #   accuracy = 0
          #   model.eval()
          #   with torch.no_grad():
          #     for inputs, labels in valid_loader:
          #       inputs, labels = inputs.to(device), labels.to(device)
          #       logps = model.forward(inputs)
          #       batch_loss = criterion(logps, labels.reshape(-1,1).float())
          #       test_loss += batch_loss.item()

          #       ps = torch.sigmoid(z)
          #       top_p, top_class = ps.topk(1, dim=1)
          #       equals = top_class == labels.view(*top_class.shape)
          #       accuracy += torch.mean(equals.squeeze().type(torch.FloatTensor)).item()

            


          # count+=1
          
          

           
      acc = (correct/total_predictions)*100.0
      running_loss /= len(train_loader)
      end_time = time.time()
      print('Training Loss: ', running_loss, 'Time: ',round(end_time - start_time,3), 's')
      print('Training Accuracy: ', round(acc,3), '%')
      # model.eval()  # switch model to the evaluation mode
      # val_preds = torch.zeros((Batch_size, 1), dtype=torch.float32, device=device)
      # with torch.no_grad():  # Do not calculate gradient since we are only predicting
      #     # Predicting on validation set
      #     for val_batch in enumerate(valid_loader):
      #         x_val,y_val = train_img.to(device), train_targ.to(device)
      # #         x_val[1] = torch.tensor(x_val[1], device=device, dtype=torch.float32)
      # #         y_val = torch.tensor(y_val, device=device, dtype=torch.float32)
      #         z_val = model(x_val)
      #         val_pred = torch.sigmoid(z_val)
      # #         val_preds[j*x_val[0].shape[0]:j*x_val[0].shape[0] + x_val[0].shape[0]] = val_pred
      #         print()
      #     val_acc = accuracy_score(y_val.cpu(), torch.round(val_preds.cpu()))
      #     val_roc = roc_auc_score(y_val.cpu(), val_preds.cpu())
          
      #     print('Epoch {:03}: | Loss: {:.3f} | Train acc: {:.3f} | Val acc: {:.3f} | Val roc_auc: {:.3f} | Training time: {}'.format(
      #     epoch + 1, 
      #     epoch_loss, 
      #     train_acc, 
      #     val_acc, 
      #     val_roc, 
      #     str(datetime.timedelta(seconds=time.time() - start_time))[:7]))
          
      #     scheduler.step(val_roc)
      #     # During the first iteration (first epoch) best validation is set to None
      #     if not best_val:
      #         best_val = val_roc  # So any validation roc_auc we have is the best one for now
      #         torch.save(model, model_path)  # Saving the model
      #         continue
              
      #     if val_roc >= best_val:
      #         best_val = val_roc
      #         patience = es_patience  # Resetting patience since we have new best validation accuracy
      #         torch.save(model, model_path)  # Saving current best model
      #     else:
      #         patience -= 1
      #         if patience == 0:
      #             print('Early stopping. Best Val roc_auc: {:.3f}'.format(best_val))
      #             break

0
40
80
120
160
200
240
280
320


In [68]:
(y.cpu()==pred.squeeze().cpu()).sum()

tensor(4096)

In [72]:
y.squeeze().shape

torch.Size([64])

In [71]:
pred.squeeze().shape

torch.Size([64])

In [85]:
y.shape

torch.Size([64, 1])

In [ ]:
0 1 0 0 0  0  == [0 1 1 0 0 ]